In [ ]:
%load_ext ext.stackql

In [ ]:
%run includes/google-audit-setup.ipynb

In [ ]:
## notebook variables
org_id = "12345"

In [ ]:
#
# discover folders and projects
#
def get_projects_query(entity_id):
    return f"""
    SELECT displayName, name, parent, projectId 
    FROM google.cloudresourcemanager.projects 
    WHERE parent = '{entity_id}';
    """

def get_folders_query(entity_id):
    return f"""
    SELECT displayName, name, parent 
    FROM google.cloudresourcemanager.folders 
    WHERE parent = '{entity_id}';
    """

resources_df, projects_df, projects, tree = get_all_resources(get_projects_query, get_folders_query)
tree

In [ ]:
#
# get project iam bindings
#
queries = [
    f"""
    SELECT role, condition, members, '{project}' as project
    FROM google.cloudresourcemanager.projects_iam_policies
    WHERE projectsId = '{project}'
    """
    for project in projects
]

project_bindings_df = get_project_bindings(queries)

In [ ]:
#
# get zones and regions
#
queries = [
    f"""
    SELECT name, SPLIT_PART(region, '/', 9) as region
    FROM google.compute.zones
    WHERE project = '{project}'
    """
    for project in projects
]

regions_df, zones_df, regions, zones, projects_regions, projects_zones = get_all_regions_and_zones(projects_df, queries)